In [2]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 8.3 kB/s 
     |████████████████████████████████| 198 kB 54.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8269e9fda62d0f2054ed1525ba9c5962b7b98037b7c04853143948a90de8acaf
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [3]:
import pandas as pd
from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()

In [4]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [5]:
spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# 창혁님이 작업중이던 파일을 넘겨받음
# 여느때처럼 spark를 이용하려 했는데 컬럼에 특수문자(:, 공백)가 있어서 그런지 에러가 떴다.
# 갑자기 분위기 pandas
df = pd.read_parquet("/content/drive/MyDrive/data/day-of-week_per_time.parquet")

In [7]:
# 시간별 데이터를 모두 더해 col에 저장한다.
sum_column = df["06:00 이전"] + df["06:00 ~ 07:00"] + df["07:00 ~ 08:00"] + df["08:00 ~ 09:00"] + \
            df["09:00 ~ 10:00"] + df["10:00 ~ 11:00"] + df["11:00 ~ 12:00"] + df["12:00 ~ 13:00"] + \
            df["13:00 ~ 14:00"] + df["14:00 ~ 15:00"] + df["15:00 ~ 16:00"] + df["16:00 ~ 17:00"] + \
            df["17:00 ~ 18:00"] + df["18:00 ~ 19:00"] + df["19:00 ~ 20:00"] + df["20:00 ~ 21:00"] + \
            df["21:00 ~ 22:00"] + df["22:00 ~ 23:00"] + df["23:00 이후"]

In [8]:
# 위에서 저장했던 col을 기존 df에 이어붙인다.
df["일간이용자수"] = sum_column

In [9]:
df

,호선,역명,요일,구분,06:00 이전,06:00 ~ 07:00,07:00 ~ 08:00,08:00 ~ 09:00,09:00 ~ 10:00,10:00 ~ 11:00,11:00 ~ 12:00,12:00 ~ 13:00,13:00 ~ 14:00,14:00 ~ 15:00,15:00 ~ 16:00,16:00 ~ 17:00,17:00 ~ 18:00,18:00 ~ 19:00,19:00 ~ 20:00,20:00 ~ 21:00,21:00 ~ 22:00,22:00 ~ 23:00,23:00 이후,일간이용자수
0,1호선,동대문,Monday,승차,488.0,331.0,524.0,688.0,588.0,504.0,555.0,633.0,643.0,677.0,707.0,746.0,787.0,818.0,548.0,423.0,399.0,313.0,128.0,10500.0
1,1호선,동대문,Monday,하차,80.0,247.0,438.0,923.0,819.0,655.0,680.0,655.0,646.0,643.0,629.0,572.0,594.0,648.0,730.0,458.0,361.0,330.0,366.0,10474.0
2,1호선,동대문,Tuesday,승차,500.0,338.0,525.0,700.0,598.0,500.0,550.0,627.0,641.0,667.0,707.0,750.0,797.0,828.0,553.0,432.0,415.0,332.0,134.0,10594.0
3,1호선,동대문,Tuesday,하차,78.0,241.0,437.0,919.0,808.0,651.0,683.0,647.0,637.0,632.0,614.0,567.0,596.0,657.0,712.0,454.0,359.0,336.0,375.0,10403.0
4,1호선,동대문,Wednesday,승차,508.0,333.0,522.0,691.0,594.0,504.0,542.0,629.0,647.0,688.0,708.0,757.0,803.0,825.0,544.0,451.0,416.0,326.0,138.0,10626.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451,8호선,천호(풍납토성),Thursday,하차,63.0,387.0,586.0,1133.0,846.0,570.0,555.0,603.0,667.0,686.0,873.0,1035.0,1453.0,2639.0,1898.0,952.0,864.0,719.0,479.0,17008.0
3452,8호선,천호(풍납토성),Friday,승차,405.0,641.0,1659.0,2328.0,1225.0,630.0,583.0,614.0,643.0,614.0,664.0,761.0,1006.0,1222.0,773.0,651.0,511.0,423.0,176.0,15529.0
3453,8호선,천호(풍납토성),Friday,하차,60.0,377.0,564.0,1106.0,876.0,597.0,582.0,637.0,707.0,727.0,943.0,1097.0,1617.0,2874.0,2136.0,1016.0,875.0,757.0,571.0,18119.0
3454,8호선,천호(풍납토성),holiday,승차,215.0,240.0,326.0,466.0,535.0,476.0,499.0,558.0,594.0,580.0,604.0,623.0,638.0,556.0,465.0,473.0,391.0,283.0,109.0,8631.0


In [10]:
# 기존 df의 시간별 이용자수를 기록한 col은 이제 필요가 없다.
regression_prepare = df[["호선", "역명", "요일", "구분", "일간이용자수"]]

In [11]:
regression_prepare

,호선,역명,요일,구분,일간이용자수
0,1호선,동대문,Monday,승차,10500.0
1,1호선,동대문,Monday,하차,10474.0
2,1호선,동대문,Tuesday,승차,10594.0
3,1호선,동대문,Tuesday,하차,10403.0
4,1호선,동대문,Wednesday,승차,10626.0
...,...,...,...,...,...
3451,8호선,천호(풍납토성),Thursday,하차,17008.0
3452,8호선,천호(풍납토성),Friday,승차,15529.0
3453,8호선,천호(풍납토성),Friday,하차,18119.0
3454,8호선,천호(풍납토성),holiday,승차,8631.0


In [12]:
group_by_station = regression_prepare.groupby(by=["역명", "호선"]).sum()

In [13]:
group_by_station

일간이용자수
역명          호선           
가락시장        3호선   95538.0
            8호선   88772.0
가산디지털단지     7호선  477826.0
강남          2호선  895439.0
강남구청        7호선  174392.0
...                   ...
홍제          3호선  180403.0
화곡          5호선  291719.0
화랑대(서울여대입구) 6호선  107029.0
회현(남대문시장)   4호선  252052.0
효창공원앞       6호선   69778.0

[288 rows x 1 columns]

In [14]:
# 개찰구가 1개인 환승역을 조사하기 위해, 일간이용자가 10명 미만인 역들을 추려내겠다.
strange_data = group_by_station[group_by_station["일간이용자수"]<10]

In [15]:
# 이상치를 가진 역의 목록을 조회해보자.
# 3개 밖에 없다. 상상보다 적은 숫자 ㄴㅇㄱ
strange_data

,,일간이용자수
역명,호선,
신내,6호선,0.0
연신내,6호선,0.0
충무로,3호선,0.0


In [16]:
# 위의 결과가 3개밖에 없는 것에 대한 내용
# 원본데이터는 동대문, 동묘앞, 서울역, 시청, 신설동, 제기동, 종각, 종로3가, 종로5가, 청량리 역만이 1호선에 속한다. 
# 원본데이터에 누락된 1호선역들 중 나머지 2~8호선과의 환승역도 추가해 기록했다.
strange_station = ["충무로", "신내", "연신내", "온수", "신도림", "신길", "석계", "창동", "도봉산", "금정"]

In [17]:
# 위의 환승역들을 따로 추려낸 df를 만든다.
df_strange_station = df[df["역명"].isin(strange_station)]

In [18]:
# 필요한 칼럼만 추린다.
df_strange_station = df_strange_station[["호선", "역명", "요일", "구분", "일간이용자수"]]

In [19]:
# 각 역이 속하는 노선에 대한 집계를 살펴봄
df_strange_station = df_strange_station.groupby(by=["역명", "호선"]).sum()

In [20]:
df_strange_station

일간이용자수
역명  호선           
도봉산 7호선  106150.0
석계  6호선  133460.0
신길  5호선   42443.0
신내  6호선       0.0
신도림 2호선  515643.0
연신내 3호선  394124.0
    6호선       0.0
창동  4호선  287385.0
충무로 3호선       0.0
    4호선  280347.0

|역명|비고|
|--|--|
|충무로|4호선 & 3호선의 환승역. 3호선 누락|
|신내|6호선 & 경춘선 환승역. 6호선이 누락(시간별 데이터 집계불가)|
|연신내|3호선 & 6호선의 환승역. 6호선 누락|
|온수|존재하지 않음|
|신도림|2호선 & 1호선의 환승역. 1호선 누락|
|신길|5호선 & 1호선의 환승역. 1호선 누락|
|석계|6호선 & 1호선의 환승역. 1호선 누락|
|창동|4호선 & 1호선의 환승역. 1호선 누락|
|도봉산|7호선 & 1호선의 환승역. 1호선 누락|
|금정|존재하지 않음|



In [21]:
# 전해 받은 데이터에 누락된 역이 확인되었다. 창혁님의 실수인지 아닌지 조사에 들어가자
not_exist = ["온수", "금정"]

In [22]:
# 온수역과 금정역은 전달받은 df에 아에 존재하지 않았음. 원본 데이터를 살펴봄
check_not_exist = pd.read_csv("/content/drive/MyDrive/data/원본/시간별집계/2021_seoul_metro.csv", encoding='euc-kr')

In [23]:
# 원본데이터에도 없는지 확인 => 창혁님은 죄가 없었다.
check_not_exist[check_not_exist["역명"].isin([not_exist])]

,날짜,호선,역번호,역명,구분,6시이전,6시-7시,7시-8시,8시-9시,9시-10시,10시-11시,11시-12시,12시-13시,13시-14시,14시-15시,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시이후


데이터 처리과정에서 일어난 오류가 아님을 확인했으므로, 이제 회귀에 사용할 각 노선별 이용자수 합계를 구해보자.<br>
이 때, strange_station의 역들은 집계에서 제외할 것이다.

In [24]:
# 이상집계된 역을 제외해줌
group_by_line = regression_prepare[regression_prepare["역명"].isin([strange_station])==False]

In [25]:
# 남은 역들을 노선별로 묶어주어 이용자수를 더해줌
group_by_line = group_by_line.groupby(by="호선").sum()

In [26]:
group_by_line

,일간이용자수
호선,
1호선,2299394.0
2호선,14500574.0
3호선,5213219.0
4호선,5173681.0
5호선,6478343.0
6호선,3303619.0
7호선,7154546.0
8호선,1918745.0


일련의 과정에서 우리는 원본 데이터에 누락된 역이 존재한다는 것을 알아냈다.<br>
단일노선의 경우는 딱히 문제가 되지 않지만, 신도림 역과 같은 환승역 중 특정 노선에 대한 데이터 자체가 누락되었다면<br>
문제가 생긴것을 인지하지 못할 가능성이 있다.(위에서 사용한<br> strange_data = group_by_station[group_by_station["일간이용자수"]<10])<br>
코드로 걸러낼 수 없기 때문)<br>
해당 이슈를 확인하기 위해선 1~8호선에 속하는 환승역들을 모두 조사하는 방법밖에는 없다.(원본 데이터가 의심되는 상황이므로)

In [27]:
# 확인할 때의 편의를 위해 각 호선별로 list를 나눔. 이미 집계한 역은 집계하지 않음
transfer_station = ["온수", "신도림", "신길", "서울역", "시청", "종로3가", "동대문", "동묘앞", "신설동", "석계", "창동", "도봉산", "가산디지털단지", "금정"] + \
                ["까치산", "대림", "사당", "교대", "잠실", "건대입구", "왕십리", "신당", "동대문역사문화공원", "을지로4가", "을지로3가", "충정로", "합정", "영등포구청"] + \
                ["연신내", "불광", "충무로", "약수", "고속터미널", "가락시장", "오금"] + \
                ["노원", "삼각지", "이수(총신대입구)"] + \
                ["공덕", "청구", "군자", "천호"] + \
                ["태릉입구"]

In [28]:
# 환승역만 추린 df를 만듦
df_transfer_check = df[df["역명"].isin(transfer_station)]

In [29]:
# 필요한 칼럼만 추려냄
df_transfer_check = df_transfer_check[["호선", "역명", "요일", "구분", "일간이용자수"]]

In [30]:
# 각 역이 속하는 노선에 대한 집계를 살펴봄
df_transfer_check = df_transfer_check.groupby(by=["역명", "호선"]).sum()

In [31]:
# 이제 하나의 노선만 기록된 역들을 집중공략하면 된다.
df_transfer_check

일간이용자수
역명      호선           
가락시장    3호선   95538.0
        8호선   88772.0
가산디지털단지 7호선  477826.0
건대입구    2호선  378475.0
        7호선  133806.0
고속터미널   3호선  428652.0
        7호선  153168.0
공덕      5호선  151378.0
        6호선  187772.0
까치산     5호선  303947.0
노원      4호선  218240.0
        7호선  213563.0
도봉산     7호선  106150.0
동대문     1호선  123327.0
        4호선  220462.0
동묘앞     1호선   93706.0
        6호선   86481.0
불광      3호선  180912.0
        6호선   48047.0
사당      2호선  407802.0
        4호선  240861.0
삼각지     4호선   52374.0
        6호선   56006.0
서울역     1호선  435495.0
        4호선  140247.0
석계      6호선  133460.0
시청      1호선  220095.0
        2호선  237324.0
신길      5호선   42443.0
신당      2호선  148684.0
        6호선   86039.0
신도림     2호선  515643.0
신설동     1호선  148429.0
        2호선   35514.0
약수      3호선  136424.0
        6호선   37951.0
연신내     3호선  394124.0
        6호선       0.0
영등포구청   2호선  241690.0
        5호선   44477.0
오금      3호선   62374.0
        5호선   38819.0
을지로3가   2호선  218554.0
        3호선  111087.0
을지로4가   2호선  134401.0
        5호선   62858.0
종로3가    1호선  265288.0
        3호선  113304.0
        5호선  127811.0
창동      4호선  287385.0
청구      5호선   40965.0
        6호선   41784.0
충무로     3호선       0.0
        4호선  280347.0
태릉입구    6호선   60309.0
        7호선   85059.0
합정      2호선  326395.0
        6호선  134608.0

In [32]:
# 그런데 역들의 숫자가 좀 적다. 병기역명등의 이유로 집계가 안됐을수도 있으니 확인해보자
len(transfer_station)

43

In [33]:
count_transfer = df[df["역명"].isin(transfer_station)]

In [34]:
count_transfer["역명"].nunique()

32

In [35]:
count_transfer["역명"].unique()

array(['동대문', '동묘앞', '서울역', '시청', '신설동', '종로3가', '건대입구', '사당', '신당',
       '신도림', '영등포구청', '을지로3가', '을지로4가', '합정', '가락시장', '고속터미널', '불광',
       '약수', '연신내', '오금', '충무로', '노원', '삼각지', '창동', '공덕', '까치산', '신길',
       '청구', '석계', '태릉입구', '가산디지털단지', '도봉산'], dtype=object)

In [36]:
# 존재를 확인한 환승역 리스트를 저장
exsist = ['동대문', '동묘앞', '서울역', '시청', '신설동', '종로3가', '건대입구', '사당', '신당',
       '신도림', '영등포구청', '을지로3가', '을지로4가', '합정', '가락시장', '고속터미널', '불광',
       '약수', '연신내', '오금', '충무로', '노원', '삼각지', '창동', '공덕', '까치산', '신길',
       '청구', '석계', '태릉입구', '가산디지털단지', '도봉산']

In [37]:
# 확인결과 12개의 역이 누락되었다... 해당역들은 다음과 같다.
set(transfer_station) - set(exsist)

{'교대',
 '군자',
 '금정',
 '대림',
 '동대문역사문화공원',
 '온수',
 '왕십리',
 '이수(총신대입구)',
 '잠실',
 '천호',
 '충정로'}

In [38]:
# 누락된 역들 리스트 저장
omission_transfer = ['교대', '군자', '금정', '대림', '동대문역사문화공원', '온수', '왕십리', '이수(총신대입구)', '잠실', '천호', '충정로']

누락역들을 정리한 내역은 다음과 같다.

|역명|환승노선|
|--|--|
|교대| 2호선 3호선|
|군자| 5호선 7호선|
|금정| 1호선 4호선|
|대림| 2호선 7호선|
|동대문역사문화공원| 2호선 4호선 5호선|
|온수| 1호선 7호선|
|왕십리| 2호선 5호선|
|이수(총신대입구)| 4호선 7호선|
|잠실| 2호선 8호선|
|천호| 5호선 8호선|
|충정로| 2호선 5호선|

In [39]:
# 각노선에 속한 역들을 전부체크하여 누락역을 확인해보겠다.
# 1호선
# 금정역과 온수역이 없다.
df[df["호선"]=="1호선"]["역명"].unique()

array(['동대문', '동묘앞', '서울역', '시청', '신설동', '제기동', '종각', '종로3가', '종로5가',
       '청량리(서울시립대입구)'], dtype=object)

In [40]:
# 2호선
# 교대 -> 교대(법원.검찰청)
# 대림 -> 대림(구로구청)
# 동대문역사문화공원 -> 동대문역사문화공원(DDP)(DDP) & 동대문역사문화공원(DDP)
# 잠실 -> 잠실(송파구청)
# 충정로 -> 충정로(경기대입구)
# 왕십리 -> 왕십리(성동구청)
df[df["호선"]=="2호선"]["역명"].unique()

array(['강남', '강변(동서울터미널)', '건대입구', '교대(법원.검찰청)', '구로디지털단지', '구의(광진구청)',
       '낙성대', '낙성대(강감찬)', '당산', '대림(구로구청)', '도림천', '동대문역사문화공원(DDP)',
       '동대문역사문화공원(DDP)(DDP)', '뚝섬', '문래', '방배', '봉천', '사당', '삼성(무역센터)',
       '상왕십리', '서울대입구(관악구청)', '서초', '선릉', '성수', '시청', '신답', '신당', '신대방',
       '신도림', '신림', '신설동', '신정네거리', '신촌', '아현', '양천구청', '역삼', '영등포구청',
       '왕십리(성동구청)', '용답', '용두(동대문구청)', '을지로3가', '을지로4가', '을지로입구', '이대',
       '잠실(송파구청)', '잠실나루', '잠실새내', '종합운동장', '충정로(경기대입구)', '한양대', '합정',
       '홍대입구'], dtype=object)

In [41]:
# 3호선
# 교대역이 존재하는지만 확인
df[df["호선"]=="3호선"]["역명"].unique()

array(['가락시장', '경복궁(정부서울청사)', '경찰병원', '고속터미널', '교대(법원.검찰청)', '구파발', '금호',
       '남부터미널(예술의전당)', '녹번', '대청', '대치', '도곡', '독립문', '동대입구', '매봉', '무악재',
       '불광', '수서', '신사', '안국', '압구정', '약수', '양재(서초구청)', '연신내', '오금', '옥수',
       '을지로3가', '일원', '잠원', '종로3가', '지축', '충무로', '학여울', '홍제'],
      dtype=object)

In [42]:
# 4호선
# 금정, 동대문역사문화공원 역이 존재하는지 확인
# 이수(총신대입구) -> 총신대입구(이수)
# 금정역이 없다.
df[df["호선"]=="4호선"]["역명"].unique()

array(['길음', '남태령', '노원', '당고개', '동대문', '동대문역사문화공원(DDP)',
       '동대문역사문화공원(DDP)(DDP)', '동작(현충원)', '명동', '미아(서울사이버대학)', '미아사거리',
       '사당', '삼각지', '상계', '서울역', '성신여대입구(돈암)', '수유(강북구청)', '숙대입구(갈월)',
       '신용산', '쌍문', '이촌(국립중앙박물관)', '창동', '총신대입구(이수)', '충무로', '한성대입구(삼선교)',
       '혜화', '회현(남대문시장)'], dtype=object)

In [43]:
# 5호선
# 동대문역사문화공원, 왕십리, 충정로 역이 존재하는지 확인
# 군자 -> 군자(능동)
# 천호 -> 천호(풍납토성)
df[df["호선"]=="5호선"]["역명"].unique()

array(['강동', '강일', '개롱', '개화산', '거여', '고덕', '공덕', '광나루(장신대)',
       '광화문(세종문화회관)', '군자(능동)', '굽은다리(강동구민회관앞)', '길동', '김포공항', '까치산',
       '답십리', '동대문역사문화공원(DDP)', '동대문역사문화공원(DDP)(DDP)', '둔촌동', '마곡', '마장',
       '마천', '마포', '명일', '목동', '미사', '발산', '방이', '방화', '상일동', '서대문', '송정',
       '신금호', '신길', '신정(은행정)', '아차산(어린이대공원후문)', '애오개', '양평', '여의나루',
       '여의도', '영등포구청', '영등포시장', '오금', '오목교(목동운동장앞)', '올림픽공원(한국체대)',
       '왕십리(성동구청)', '우장산', '을지로4가', '장한평', '종로3가', '천호(풍납토성)', '청구',
       '충정로(경기대입구)', '하남검단산', '하남시청(덕풍·신장)', '하남풍산', '행당', '화곡'],
      dtype=object)

In [44]:
# 6호선
# 6호선은 아무문제없다.
df[df["호선"]=="6호선"]["역명"].unique()

array(['고려대(종암)', '공덕', '광흥창(서강)', '구산', '녹사평(용산구청)', '대흥(서강대앞)', '독바위',
       '돌곶이', '동묘앞', '디지털미디어시티', '마포구청', '망원', '버티고개', '보문', '봉화산(서울의료원)',
       '불광', '삼각지', '상수', '상월곡(한국과학기술연구원)', '새절(신사)', '석계', '신내', '신당',
       '안암(고대병원앞)', '약수', '역촌', '연신내', '월곡(동덕여대)', '월드컵경기장(성산)', '응암',
       '이태원', '증산(명지대앞)', '창신', '청구', '태릉입구', '한강진', '합정', '화랑대(서울여대입구)',
       '효창공원앞'], dtype=object)

In [45]:
# 7호선
# 군자, 대림, 온수, 이수(총신대입구)역이 존재하는지 확인
# 특이사항: 7호선은 이수(총신대입구)역이 "이수역"이라는 이름으로 존재
# 온수 -> 온수(성공회대입구)
df[df["호선"]=="7호선"]["역명"].unique()

array(['가산디지털단지', '강남구청', '건대입구', '고속터미널', '공릉(서울과학기술대)', '광명사거리',
       '군자(능동)', '굴포천', '까치울', '남구로', '남성', '내방', '노원', '논현', '대림(구로구청)',
       '도봉산', '뚝섬유원지', '마들', '먹골', '면목', '반포', '보라매', '부천시청', '부천종합운동장',
       '부평구청', '사가정', '삼산체육관', '상도', '상동', '상봉(시외버스터미널)', '수락산',
       '숭실대입구(살피재)', '신대방삼거리', '신중동', '신풍', '어린이대공원(세종대)', '온수(성공회대입구)',
       '용마산', '용마산(용마폭포공원)', '이수', '장승배기', '장암', '중계', '중곡', '중화', '천왕',
       '철산', '청담', '춘의', '태릉입구', '하계', '학동'], dtype=object)

In [46]:
# 8호선
# 천호역이 존재하는지 확인
df[df["호선"]=="8호선"]["역명"].unique()

array(['가락시장', '강동구청', '남한산성입구(성남법원.검찰청)', '단대오거리', '모란', '몽촌토성(평화의문)',
       '문정', '복정', '산성', '석촌', '송파', '수진', '신흥', '암사', '잠실(송파구청)', '장지',
       '천호(풍납토성)'], dtype=object)

확인결과 원본데이터에 금정역은 존재하지 않았고 금정역을 제외한 나머지 10개역은 역명이 다름을 확인하였다.<br>
즉 우려했던 문제(환승역이 단일역으로 표기되는 문제)는 일어나지 않을것으로 보인다.

<h1>역명 통일시키기</h1>



In [47]:
# 노선도 이미지가 구 역명을 사용하고 있으므로 일단 구 역명을 사용하도록 함
# 마친 위에 각 노선별 역들을 추려놨으니 노선도와 대조하면서 dictionary를 만들었다.
station_need_change = {"강면(동서울터미널)": "강변", "교대(법원.검찰청)": "교대", "구의(광진구청)": "구의", "낙성대(강감찬)": "낙성대", 
                       "대림(구로구청)": "대림", "동대문역사문화공원(DDP)": "동대문역사문화공원", "동대문역사문화공원(DDP)(DDP)": "동대문역사문화공원",
                       "삼성(무역센터)": "삼성", "서울대입구(관악구청)": "서울대입구", "왕십리(성동구청)": "왕십리", "용두(동대문구청)": "용두",
                       "잠실(송파구청)": "잠실", "충정로(경기대입구)": "충정로", "경복궁(정부서울청사)": "경복궁", "남부터미널(예술의전당)": "남부터미널",
                       "동작(현충원)": "동작", "미아(서울사이버대학)": "미아", "성신여대입구(돈암)": "성신여대입구", "수유(강북구청)": "수유",
                       "숙대입구(갈월)": "숙대입구", "이촌(국립중앙박물관)": "이촌", "총신대입구(이수)": "이수(총신대입구)", "한성대입구(삼선교)": "한성대입구",
                       "회현(남대문시장)": "회현", "광나루(장신대)": "광나루", "광화문(세종문화회관)": "광화문", "군자(능동)": "군자",
                       "굽은다리(강동구민회관앞)": "굽은다리", "신정(은행정)": "신정", "아차산(어린이대공원후문)": "아차산", "오목교(목동운동장앞)": "오목교",
                       "올림픽공원(한국체대)": "올림픽공원", "천호(풍납토성)": "천호", "고려대(종암)": "고려대", "광흥창(서강)": "굉흥창",
                       "녹사평(용산구청)": "녹사평", "대흥(서강대앞)": "대흥", "봉화산(서울의료원)": "봉화산", "상월곡(한국과학기술연구원)": "상월곡",
                       "새절(신사)": "새절", "안암(고대병원앞)": "안암", "월곡(동덕여대)": "월곡", "월드컵경기장(성산)": "월드컵경기장",
                       "증산(명지대앞)": "증산", "화랑대(서울여대입구)": "화랑대", "상봉(시외버스터미널)": "상봉", "숭실대입구(살피재)": "숭실대입구",
                       "어린이대공원(세종대)": "어린이대공원", "온수(성공회대입구)": "온수", "용마산(용마폭포공원)": "용마산",
                       "남한산성입구(성남법원.검찰청)": "남한산성입구", "몽촌토성(평화의문)": "몽촌토성"
                    }

In [48]:
# 앞에서 만든 dictionary의 key값을 저장한 리스트 저장
station_need_change_list = []
for i in station_need_change:
    station_need_change_list.append(i)

In [59]:
df_change_station_name = regression_prepare.replace({"역명": station_need_change})

In [61]:
# 변경이 안된 역이 있는지 확인
df_change_station_name[df_change_station_name["역명"].isin(station_need_change_list)]

,호선,역명,요일,구분,일간이용자수


역명이 잘 바뀌었으니 이제 다시 환승역을 체크해보겠다.

In [71]:
# 노선도 내의 모든 환승역들만 추려냄
df_transfer_check = df_change_station_name[df_change_station_name["역명"].isin(transfer_station)]

In [74]:
# 어떤 역들이 추려졌는지 확인
df_transfer_check["역명"].unique()

array(['동대문', '동묘앞', '서울역', '시청', '신설동', '종로3가', '건대입구', '교대', '대림',
       '동대문역사문화공원', '사당', '신당', '신도림', '영등포구청', '왕십리', '을지로3가', '을지로4가',
       '잠실', '충정로', '합정', '가락시장', '고속터미널', '불광', '약수', '연신내', '오금', '충무로',
       '노원', '삼각지', '창동', '이수(총신대입구)', '공덕', '군자', '까치산', '신길', '천호',
       '청구', '석계', '태릉입구', '가산디지털단지', '도봉산', '온수'], dtype=object)

In [75]:
# 추려진 역들을 리스트에 저장
exsist = ['동대문', '동묘앞', '서울역', '시청', '신설동', '종로3가', '건대입구', '교대', '대림',
       '동대문역사문화공원', '사당', '신당', '신도림', '영등포구청', '왕십리', '을지로3가', '을지로4가',
       '잠실', '충정로', '합정', '가락시장', '고속터미널', '불광', '약수', '연신내', '오금', '충무로',
       '노원', '삼각지', '창동', '이수(총신대입구)', '공덕', '군자', '까치산', '신길', '천호',
       '청구', '석계', '태릉입구', '가산디지털단지', '도봉산', '온수']

In [76]:
# 빠진역을 확인. 금정역은 원본데이터에 존재하지 않는 역이다.(아무일 없었다...!)
set(transfer_station) - set(exsist)

{'금정'}

In [77]:
# groupby를 이용하여 환승역들의 집계상황을 확인해보자.
df_transfer_check = df_transfer_check.groupby(by=["역명", "호선"]).sum()

In [80]:
# 모든 row를 볼수 있게 해주는 코드.
pd.set_option('display.max_rows', None)
df_transfer_check

일간이용자수
역명        호선           
가락시장      3호선   95538.0
          8호선   88772.0
가산디지털단지   7호선  477826.0
건대입구      2호선  378475.0
          7호선  133806.0
고속터미널     3호선  428652.0
          7호선  153168.0
공덕        5호선  151378.0
          6호선  187772.0
교대        2호선  376386.0
          3호선  121359.0
군자        5호선  122961.0
          7호선  139773.0
까치산       5호선  303947.0
노원        4호선  218240.0
          7호선  213563.0
대림        2호선  270329.0
          7호선   85617.0
도봉산       7호선  106150.0
동대문       1호선  123327.0
          4호선  220462.0
동대문역사문화공원 2호선  282046.0
          4호선  310129.0
          5호선   63180.0
동묘앞       1호선   93706.0
          6호선   86481.0
불광        3호선  180912.0
          6호선   48047.0
사당        2호선  407802.0
          4호선  240861.0
삼각지       4호선   52374.0
          6호선   56006.0
서울역       1호선  435495.0
          4호선  140247.0
석계        6호선  133460.0
시청        1호선  220095.0
          2호선  237324.0
신길        5호선   42443.0
신당        2호선  148684.0
          6호선   86039.0
신도림       2호선  515643.0
신설동       1호선  148429.0
          2호선   35514.0
약수        3호선  136424.0
          6호선   37951.0
연신내       3호선  394124.0
          6호선       0.0
영등포구청     2호선  241690.0
          5호선   44477.0
오금        3호선   62374.0
          5호선   38819.0
온수        7호선  108083.0
왕십리       2호선  155048.0
          5호선   51265.0
을지로3가     2호선  218554.0
          3호선  111087.0
을지로4가     2호선  134401.0
          5호선   62858.0
이수(총신대입구) 4호선  213028.0
잠실        2호선  690823.0
          8호선  157263.0
종로3가      1호선  265288.0
          3호선  113304.0
          5호선  127811.0
창동        4호선  287385.0
천호        5호선  193123.0
          8호선  179053.0
청구        5호선   40965.0
          6호선   41784.0
충무로       3호선       0.0
          4호선  280347.0
충정로       2호선  108535.0
          5호선   50202.0
태릉입구      6호선   60309.0
          7호선   85059.0
합정        2호선  326395.0
          6호선  134608.0

따로 처리가 필요한 역들은 다음과 같다.

|역명|집계된노선|누락된노선|
|--|--|--|
|가산디지털단지|7호선|1호선|
|도봉산|7호선|1호선|
|석계|6호선|1호선|
|신길|5호선|1호선|
|신도림|2호선|1호선|
|연신내|6호선|3호선|
|온수|7호선|1호선|
|이수(총신대입구)|4호선|7호선|
|창동|4호선|1호선|
|충무로|4호선|3호선|